# CS211: Data Privacy
## In-Class Exercise, week of 8/29/2022

In [34]:
# Load the data and libraries
import pandas as pd
import numpy as np

adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

## Question 1

Construct a de-identified version of the `adult` dataset.

In [35]:
def deidentify_adult():
    return adult.drop(['Name', 'DOB', 'SSN'], axis=1)

adult_deid = deidentify_adult()

In [36]:
deidentify_adult()

,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,41328,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,94735,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,49628,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,8213,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [37]:
assert 'Name' not in adult_deid.columns
assert 'DOB' not in adult_deid.columns
assert 'SSN' not in adult_deid.columns

## Question 2

What columns did you remove from the dataset to de-identify it? Why did you choose these columns? Why did you not choose the remaining columns?

I looked at the test case and saw that I needed to remove Name, DOB, SSN.

These are PII according to HIPAA.

These properties are generally unique or close to unique for individuals

## Question 3

Write a pandas expression to return just the row containing information about Brenn McNeely.

In [38]:
def get_brenns_row():
    return adult[adult['Name'] == 'Brenn McNeely']

brenns_row = get_brenns_row()[['Name', 'DOB', 'Zip', 'Age']]

In [39]:
assert len(brenns_row) == 1
assert brenns_row['Zip'].iloc[0] == 95668
assert brenns_row['Age'].iloc[0] == 38

In [40]:
brenns_row

,Name,DOB,Zip,Age
2,Brenn McNeely,8/6/1991,95668,38


## Question 4

Conduct a linking attack to recover Brenn's data from the `adult_deid` dataset.

In [41]:
def recover_brenns_row():
    brenns_zip = int(brenns_row['Zip'])
    brenns_age = int(brenns_row['Age'])
    return adult_deid[(adult_deid['Zip'] == brenns_zip) & (adult_deid['Age'] == brenns_age)]

brenns_recovered_row = recover_brenns_row()
brenns_recovered_row

,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
2,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [42]:
assert len(brenns_recovered_row) == 1
assert brenns_recovered_row['Education'].iloc[0] == 'HS-grad'

In [58]:
brenns_recovered_row

,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
2,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [48]:
adult['Age'].max()

90

In [55]:
adult['Age'].sum() - adult[adult['Name'] != 'Brenn McNeely']['Age'].sum()

38

In [56]:
adult['Age'].mean()

38.58164675532078